# Choosing transforms for 🦠 Sartorius -  🦠 Cell Instance Segmentation 🦠 

Its generally a practice to go for robust augmentation technqiues, and using as much as possible. 
In this experiment, I am aiming to choose augmentation based on what might help model.


In this competition, since we have limited amount of trainig data, using augmentation widely might be key to winning this competition.

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import random
import torch
from pytorch_lightning import LightningDataModule
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import albumentations as A
from albumentations import Normalize
from albumentations.pytorch import ToTensorV2

## Crtieria to choose augmentation

1. Augmentations doesn't harm the image. 
2. Augmentation helps model by creating potential edge cases missing in training dataset, but also, doesn't do a over-kill to produce cases that model might never see.
3. Augmentation works fine for both Image and Mask.

In [ ]:
df_train = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")
df_train.head()

In [ ]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    '''mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)
    '''
    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)


class Segmentation_ImageDataset(Dataset):
    def __init__(self, dataframe, path_column='file_path', label_column='target',
                 transform=None , function = None) -> None:
        """dataframe - pandas dataframe with 2 columns - file_path and target."""
        super().__init__()
        self.df = dataframe
        self.transform = transform
        self.paths = self.df[path_column]
        self.function = function

        #Classes - Unique pixel values in masks

    def __getitem__(self, index):
        
        image_id = df_train['id'][index]
        labels = df_train[df_train["id"] == image_id]["annotation"].tolist()
        
        colors = False #True

        if colors:
            mask = np.zeros((520, 704, 3))
            for label in labels:
                mask += rle_decode(label, shape=(520, 704, 3), color=np.random.rand(3))
        else:
            mask = np.zeros((520, 704, 1))
            for label in labels:
                mask += rle_decode(label, shape=(520, 704, 1))
        mask = mask.clip(0, 1)
        

        image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.function != None:
            image = self.function(image)


        if self.transform is not None:
            tranformed = self.transform(image=image , mask=mask)
            tranformed_image = tranformed['image']
            tranformed_mask = tranformed['mask']

        return image , mask , tranformed_image ,  tranformed_mask

    def __len__(self):
        return len(self.paths)


def image_batch_comparison_visualizer2d(dataset, count=24, subplot=(6, 4),  unnormalize = True ,
                               cmap='gist_gray', random_img=True,
                             figheight=10, figwidth=20   , alpha = 0.1 , title = True):
    fig = plt.figure(figsize=(figwidth, figheight))

    #dataset.df = shuffle(dataset.df)
    images_shown = 0

    while images_shown < count-2:
        if random_img == True:
            index = random.randint(0, dataset.df.shape[0] -1  )

        image , mask , transformed_image ,  transformed_mask = dataset[index]
        image = image #.numpy().transpose((1, 2, 0))
        transformed_image = transformed_image.numpy().transpose((1, 2, 0))
        mask = mask #.numpy() #.transpose((1, 2, 0))
        transformed_mask = transformed_mask.numpy() #.transpose((1, 2, 0))


        if unnormalize == True:
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            transformed_image = std * transformed_image + mean
            transformed_image = np.clip(transformed_image, 0, 1)
            
        
        
        plt.subplot(subplot[0], subplot[1], images_shown + 1)
        plt.axis('off')
        plt.tight_layout()
        plt.imshow(image )
        if title == True: 
            plt.title('Image')

        plt.subplot(subplot[0], subplot[1], images_shown + 2)
        plt.axis('off')
        plt.tight_layout()
        plt.imshow(image )
        plt.imshow(mask, alpha=alpha)
        if title == True: 
            plt.title('Image + Mask Overlap')

        plt.subplot(subplot[0], subplot[1], images_shown + 3)        
        plt.axis('off')
        plt.tight_layout()
        plt.imshow(transformed_image , cmap=cmap)
        if title == True: 
            plt.title('Tansformed Image')
        
        plt.subplot(subplot[0], subplot[1], images_shown + 4)        
        plt.axis('off')
        plt.tight_layout()
        plt.imshow(transformed_image , cmap=cmap)
        plt.imshow(transformed_mask , alpha = alpha)
        if title == True: 
            plt.title('Tansformed Image + Mask  Overlap')

        images_shown += 4
    plt.show()
    
def original_image_batch_visualizer2d(dataset, plot_mask =True ,  count=24, subplot=(6, 4),  unnormalize = True ,
                               cmap='gist_gray', random_img=True,
                             figheight=10, figwidth=20   , alpha = 0.1):
    fig = plt.figure(figsize=(figwidth, figheight))
    images_shown = 1

    while images_shown < count + 1 :
        if random_img == True:
            index = random.randint(0, dataset.df.shape[0] -1  )

        image , mask , transformed_image ,  transformed_mask = dataset[index]
        image = image #.numpy().transpose((1, 2, 0))
        mask = mask #.numpy() #.transpose((1, 2, 0))
        
        
        plt.subplot(subplot[0], subplot[1], images_shown )
        plt.axis('off')
        plt.tight_layout()
        plt.imshow(image )
        if plot_mask == True:
            plt.imshow(mask, alpha=alpha)

        images_shown += 1
    plt.show()
    

# Lets look at the Images

Aim is to look at enough samples, with and without masks, and get a fair idea of the distribution.

For this, we wont use any augmentation, only the basc -Resize and Normalize.

In [ ]:
count = 100
subplot = (10,10)
width = 18 * subplot[0]/  subplot[1] + 2
height = 15 * subplot[1]/  subplot[0]

image_size = 224
imagenet_stats = {"mean": [0.485, 0.456, 0.406],
                      "std": [0.229, 0.224, 0.225]}
    
valid_tfms_albu = A.Compose(
        [A.Resize(image_size, image_size),
         Normalize(mean= imagenet_stats['mean'] ,std= imagenet_stats['std'] ,),
         ToTensorV2() ,])

train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu)

original_image_batch_visualizer2d( train_dataset,plot_mask =False ,  count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.15)

In [ ]:
original_image_batch_visualizer2d( train_dataset,plot_mask =True ,  count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.25)

### Looking at original images, we observe, there is little to no variation in 
1. Hue
2. Brightness
3. Contrast

### Looking into Masks overlap, we observe - 
1. Different levels of zooms
2. Original data also has several cells(on margins) that are cropped partially, so artifically RandomResizedCrop might not harm our data.
3. Cells have no orientation what so ever, so Rotation is going to helpful.

## Based on this, lets choose few basic augmentations to start with - 

1. RandomResizedCrop
2. RandomRotation
3. RandomRotate90
3. RandomScale #Zoom in and out
4. Flip
5. Normalize
6. ToTensor




In [ ]:
p = 1
count = 16 
subplot = (4,4)
width = 15 * subplot[0]/  subplot[1] + 5
height = 15 * subplot[1]/  subplot[0]

valid_tfms_albu = A.Compose(
        [#A.Resize(image_size, image_size),
         A.RandomScale((0.8 , 1.5)),
         A.Rotate(limit=45) , 
         A.RandomResizedCrop(image_size, image_size, scale=(0.9, 1), p=p),
         A.Flip(p=p),
        A.RandomRotate90(p=p),    
         Normalize(mean= imagenet_stats['mean'] ,std= imagenet_stats['std'] ,),
         ToTensorV2() ,])
    
train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu)

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)

##  Now lets consider other methods - 

Lets add RandomBrightnessContrast, Hue saturation and cutouts.

In [ ]:
 valid_tfms_albu = A.Compose([
            A.Resize(image_size, image_size),
            A.OneOf([ A.RandomBrightnessContrast( brightness_limit=0.2, contrast_limit=0.2,),
                    A.HueSaturationValue( hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50),], p=p,),
            A.Cutout(max_h_size=int(image_size * 0.05), max_w_size=int(image_size * 0.05), num_holes=5, p= 0.5),
            Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
            ToTensorV2()
            ])
    
train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu)

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)

Since Cutouts dont seem to be applied to masks, it seems they wont help much in understanding pixel level information.

ShiftScale seems to generate intersting results. This can be added to out list.

# Geometrical transformations

Now, lets see if we can use geometrical transformations.
But, before that, lets add grid to the image.


In [ ]:
def draw_grid(image, grid_size = 50):
    for i in range(0, image.shape[1], grid_size):
        image = cv2.line(image, (i, 0), (i, image.shape[0]), color=(255,))
    for j in range(0, image.shape[0], grid_size):
        image = cv2.line(image, (0, j), (image.shape[1], j), color=(255,))
    return image  

### Lets start by adding Elastic transform and Persepective.

In [ ]:
valid_tfms_albu = A.Compose([
    A.ElasticTransform ( p = 1) ,
    A.Perspective (scale=(0.05, 0.1) , p = 1),
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
            ToTensorV2()])
    
train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu ,
                                         function = draw_grid )

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)

### PieceWise Affine and GridDistort

In [ ]:
valid_tfms_albu = A.Compose([
     A.PiecewiseAffine (scale=(0.03, 0.05) , p = 1),
     A.GridDistortion(p=1),
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
            ToTensorV2()
            ])

    
train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu ,
                                         function = draw_grid )


image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)

### Optical distortion 

This one is slightly tricky. Over-doing it might distort the scans too much.

In [ ]:
valid_tfms_albu = A.Compose([
     A.OpticalDistortion(distort_limit=0.5, shift_limit=0.5, p=1) ,
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
            ToTensorV2()])

    
train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu ,
                                         function = draw_grid )

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)

#### We can use most of these geometrical augmentations work, we will include them in list.

### Lets try other methods now,

### CLAHE augmentation
CLAHE, Coarse dropout

In [ ]:
valid_tfms_albu = A.Compose([
            A.Resize(image_size, image_size),
            A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.4, rotate_limit=45, p=p), #Randomly apply affine transforms: translate, scale and rotate the input.
            A.CLAHE(clip_limit=(1, 8), p=p),
            A.CoarseDropout(max_holes=10, p=p),
            Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
            ToTensorV2()
            ])

train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , transform=valid_tfms_albu,
                                                        function = draw_grid)

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3)


Using CLAHE seems to improve visualisation of cells. This can be used.

# Finally, lets combine the them.

We will use one of the geometrical optionally, and set probability of most transforms as 0.25.

In [ ]:
p = 0.25

valid_tfms_albu = A.Compose(
        [A.Resize(image_size+25, image_size+25),
        A.RandomScale((0.8 , 1.5) , p = p ),
        A.Rotate(limit=45 , p = p ) , 
        A.RandomResizedCrop(image_size, image_size, scale=(0.9, 1), p=1),
        A.Flip(p=p),
        A.RandomRotate90(p=p),    

        A.OneOf([ A.ElasticTransform () ,
                  A.Perspective (scale=(0.05, 0.1) ),
                  A.PiecewiseAffine (scale=(0.03, 0.05) ),
                 A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.4, rotate_limit=45),],
                p=p,),
        A.OneOf([ A.GridDistortion() ,
                  A.OpticalDistortion(distort_limit=0.5, shift_limit=0.5) ],
                p=p,),
        
        A.CLAHE(clip_limit=(1, 8), p=p),
        A.CoarseDropout(max_holes=10, p=p),
        A.Cutout(max_h_size=int(image_size * 0.05), max_w_size=int(image_size * 0.05), num_holes=5, p= p),
        
        Normalize(mean=imagenet_stats['mean'], std=imagenet_stats['std'], ),
        ToTensorV2()
            ])

count = 4 * 15
subplot = (15,4)
width = 7  #* subplot[0]/  subplot[1] + 5
height = 30 #* subplot[1]/  subplot[0]

train_dataset = Segmentation_ImageDataset(dataframe= df_train , path_column= 'id' , 
                                          transform=valid_tfms_albu, function = draw_grid)

image_batch_comparison_visualizer2d( train_dataset, count= count, subplot= subplot,   unnormalize =  True,
                        figheight=height, figwidth=width , alpha = 0.3 , title = False)

# More info on Albumentation package - 

https://albumentations.ai/docs/examples/example_kaggle_salt/